### SRIJAN BHUSHAN MDS202039
### ANUJA PAL MDS202006

Importing the packages required:

In [ ]:
import numpy as np
import pandas as pd

original_parity takes in an array as an input and returns 1 if number of -1's is even, and 0 otherwise.

In [ ]:
# function in which we are trying to predict

# True / 1 means parity is even
# False / 0 means parity is odd
def original_parity(arr):
  """
  input: np.array (consisting of 1 or -1 as elements)
  output: boolean

  **ASSUMED: input length always even**

  return True / 1 if number of -1s is even
  return False / 0 if number of -1s is odd
  """
  return int(arr.prod() == 1)

data_gen is simply to help us create a data-frame where we will have 'n' columns, the 'i'th column gives us the value of the 'i'th component of the vector.
The last column will be the label i.e. if parity is even -> 1, if parity is odd -> 0.

In [ ]:
def data_gen(n_vectors, vector_len=64):
  assert vector_len%2 == 0
  """
  input: n_vectors (int), vector_len (int)
  output: pd dataframe

  returns a dataframe with columns as entries in i^{th} position of a supposed array
  and an extra column 'y' which returns the orginal_parity() output of the supposed array
  as input.
  """

  #list of vectors
  data = []

  #list of outputs
  outp = []

  # generation
  for i in range(n_vectors):
    generated_vector = np.random.choice([-1,1], size=vector_len)
    obtained_parity = original_parity(generated_vector)
    data.append(generated_vector)
    outp.append(obtained_parity)
  
  # convert to np array for easier use
  data = np.array(data)

  # dataframe creation
  df_dict = {
      'x'+str(i):data[:,i] for i in range(vector_len)
  }
  df_dict['y'] = outp

  # return dataframe
  return pd.DataFrame(df_dict)
  


In [ ]:
# small test contributing towards correctness
data_gen(15, 6)

,x0,x1,x2,x3,x4,x5,y
0,-1,-1,1,-1,1,1,0
1,-1,1,1,-1,1,1,1
2,1,-1,1,-1,1,-1,0
3,1,1,1,-1,1,1,0
4,-1,1,-1,1,1,1,1
5,1,-1,1,1,-1,1,1
6,-1,-1,1,1,1,1,1
7,-1,1,1,1,-1,-1,0
8,-1,-1,-1,-1,1,-1,0
9,-1,-1,-1,-1,1,-1,0


Generating the datasets, one of size 2000 i.e. 2000 vectors of length 64, each a sequence of 1's and -1's, along with their corresponding labels for parity.
The other is of size 5000.

In [ ]:
# generating data on which we will test our self-made NN
test_data_2k = data_gen(2000) # each vector is of len 64
test_data_5k = data_gen(5000) # each vector is of len 64

Here, we separate the parity labels from the data of vectors.

In [ ]:
# splitting the generated data
input_2k = test_data_2k[['x'+str(i) for i in range(64)]]
input_5k = test_data_5k[['x'+str(i) for i in range(64)]]
output_2k = test_data_2k['y']
output_5k = test_data_5k['y']

Structure of our neural network:

In [ ]:
"""
NN as follows: (2 hidden layers; {input_len} nodes in first hidden layer then 1 node in next hidden layer)

(assume 2 inputs)

[ input layer]       [bias = 0]                              [bias = 0]

{inp1}             --- (weight=1) ---> (node1; activation f1) ---> (node3; weight=1) \
                                                                                      ---> (node3;activation f2) ---> (output)
{inp2}             --- (weight=1) ---> (node2; activation f1) ---> (node3; weight=1) /

activations:
f1 input(x) ={ int( x == 1 ) }
f2 input([x1,...xn]) ={ int(sum(xi)%2 == 0) }
"""


Activation functions:

In [ ]:
# Building the activation functions
def f1(x):
  """
  input: int
  output: int
  
  return 1 if x is 1
  return 0 else
  """
  return int(x == 1)

def f2(arr):
  """
  input: np.array
  output: int

  return 1 if sum(arr) is even
  return 0 else
  """
  return int(arr.sum() %2 == 0)

Constructing the neural network:

In [ ]:
# under construction
def predefined_NN_gen(inp_size=64):
  """
  input: int
  output: object of <class NN>

  generates a parity finding neural network
  consisting of 2 hidden layers and input layer
  contains {inp_size} nodes
  """
  class NN():
    def __init__(self):
      self.inp_size = inp_size
      self.weights1 = np.eye(inp_size)
      self.f1 = f1
      self.weights2 = np.ones(inp_size)
      self.f2 = f2
      return 

    def feedforward(self,row):
      # layer 1 computations
      o1 = np.zeros(self.inp_size)

      for i in range(self.inp_size):
        node_i_input = np.dot(row, self.weights1[i,:])
        node_i_activation = self.f1(node_i_input)
        o1[i] = node_i_activation

      # layer 1 returns values into o1

      # layer 2 computations
      return self.f2(np.dot(self.weights2, o1))
    

    def predict(self, df):
      inputs = df.to_numpy()
      outputs = np.zeros(inputs.shape[0])
      for i in range(inputs.shape[0]):
        outputs[i] = self.feedforward(inputs[i,:])
      
      return outputs

  return NN()

In [ ]:
nn = predefined_NN_gen()

Accuracy scores on the 2k and 5k datasets:

In [ ]:
# accuracy on 2k dataset
np.mean(nn.predict(input_2k) == output_2k)

1.0

In [ ]:
# accuracy on 5k dataset
np.mean(nn.predict(input_5k) == output_5k)

1.0

We have 100% accuracy on our predictions
In fact, our self made neural network will always produce the correct answer.